<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/VideoLlaMA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q install decord ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 82.6 MB/s eta 0:00:00


In [3]:
!pip -q uninstall transformers -y
!pip -q install git+https://github.com/huggingface/transformers


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from huggingface_hub import hf_hub_download

# Download the video (10s, 300 frames at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo",          # Dataset repo ID
    filename="eating_spaghetti.mp4",      # Filename to fetch
    repo_type="dataset"                   # Important: it's a dataset, not a model
)

print("Downloaded video to:", file_path)


Downloaded video to: /root/.cache/huggingface/hub/datasets--nielsr--video-demo/snapshots/657fbe79a427948714ba0f45f454a494d346f75e/eating_spaghetti.mp4


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model_name = "DAMO-NLP-SG/VideoLLaMA3-7B"
model_name = "DAMO-NLP-SG/VideoLLaMA3-2B"

# Load the model and processor
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
# )

# if no flash attention

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",  # ← Alternative here
    trust_remote_code=True,
)

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

# Replace with your actual video file path
video_path = file_path
question = "Describe this video in detail."

# Prepare the conversation
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 8}},
            {"type": "text", "text": question},
        ],
    },
]

# Preprocess the inputs
inputs = processor(conversation=conversation, return_tensors="pt")
# Move all tensors to GPU if available
inputs = {k: (v.to(device) if isinstance(v, torch.Tensor) else v) for k, v in inputs.items()}
# Cast pixel values for efficiency
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

# Run generation
output_ids = model.generate(**inputs, max_new_tokens=128)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print("Assistant:", response)

ImportError: cannot import name 'VideoInput' from 'transformers.image_utils' (/usr/local/lib/python3.11/dist-packages/transformers/image_utils.py)